# SK Memory - RAG Pattern Primer

Learning objectives:

- Setting up SK to save memories
- Saving, retrieving, recalling, and deleting memories

Reference:
- https://github.com/Azure-Samples/azure-search-openai-demo
- https://github.com/azure-samples/semantic-kernel-rag-chat

**Important:**

- A memory includes at a minium a key, some text (a text chunk), and the embedding
- The embedding is used to perform a semantic search, but the text is what is used for augmentation

## Load the required .NET packages and supporting classes

In [47]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta8"
#r "nuget: dotenv.net"

//using System.ComponentModel;
//using System.Net.Http;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI.TextEmbedding;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Plugins.Memory;

using dotenv.net;


const string MemoryCollectionName = "aboutMe";

Installed Packages dotenv.net, 3.1.3 Microsoft.SemanticKernel, 1.0.0-beta8

## Load the OpenAI variables from environment variables or an .env file

In [48]:
DotEnv.Load();
var deploymentName = Environment.GetEnvironmentVariable("GPT_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("GPT_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");
var adaDeploymentName = "ada";
Console.WriteLine($"Using deployment: {deploymentName} at: {endpoint} with key {apiKey.Substring(0, 5)}...");

Using deployment: gpt at: https://alemorfr.openai.azure.com/ with key 32c87...


## Get a Kernel with an embedding service

In [49]:
var ramStore = new VolatileMemoryStore();

var kernel = new KernelBuilder()
            .WithAzureOpenAIChatCompletionService(deploymentName, endpoint, apiKey)
            .WithAzureOpenAITextEmbeddingGenerationService("ada", endpoint, apiKey)
            .Build();
            
var embeddingGenerator = new AzureOpenAITextEmbeddingGeneration(adaDeploymentName, endpoint, apiKey,null,null,null);
SemanticTextMemory textMemory = new(ramStore, embeddingGenerator);

## Save memories - Method 1

In [50]:
// ========= Store memories using the kernel =========

await textMemory.SaveInformationAsync(MemoryCollectionName, id: "info1", text: "My name is Andre");
await textMemory.SaveInformationAsync(MemoryCollectionName, id: "info2", text: "I work as a tourist operator");
await textMemory.SaveInformationAsync(MemoryCollectionName, id: "info3", text: "I've been living in Seattle since 2005");
await textMemory.SaveInformationAsync(MemoryCollectionName, id: "info4", text: "I visited France and Italy five times since 2015");

## Save memories - Method 2 - using a skill

In [51]:
// ========= Store memories using semantic function =========

// Add Memory as a skill for other functions
var memoryPlugin = new TextMemoryPlugin(textMemory);
var memoryFunctions = kernel.ImportFunctions(memoryPlugin);

// Build a semantic function that saves info to memory
const string SaveFunctionDefinition = "{{save $info}}";
var memorySaver = kernel.CreateSemanticFunction(SaveFunctionDefinition);

var result = await kernel.RunAsync(memoryFunctions["Save"], new ContextVariables()
        {
            [TextMemoryPlugin.CollectionParam] = MemoryCollectionName,
            [TextMemoryPlugin.KeyParam] = "info5",
            ["input"] = "My family is from New York"
        });

Console.WriteLine(result);

Microsoft.SemanticKernel.Orchestration.KernelResult


## Retrive a memory by Key

In [52]:
// ========= Test memory remember =========
Console.WriteLine("========= Example: Recalling a Memory =========");

var result = await kernel.RunAsync(memoryFunctions["Retrieve"], new ContextVariables()
        {
            [TextMemoryPlugin.CollectionParam] = MemoryCollectionName,
            [TextMemoryPlugin.KeyParam] = "info5"
        });
Console.WriteLine(result);

========= Example: Recalling a Memory =========
My family is from New York


## Recall a memory based on relevance and a count limit

In [53]:
Console.WriteLine("========= Example: Recalling an Idea =========");

 await foreach (var answer in textMemory.SearchAsync(
            collection: MemoryCollectionName,
            query: "where did I grow up?",
            limit: 2,
            minRelevanceScore: 0.79,
            withEmbeddings: true))
        {
            Console.WriteLine($"Answer: {answer.Metadata.Text}");
        }

========= Example: Recalling an Idea =========
Answer: My family is from New York
Answer: I've been living in Seattle since 2005


## Recall a memory using an SK function

In [54]:
Console.WriteLine("========= Example: Using Recall in a Semantic Function =========");

result = await kernel.RunAsync(memoryFunctions["Recall"], new ContextVariables()
        {
            [TextMemoryPlugin.CollectionParam] = MemoryCollectionName,
            [TextMemoryPlugin.LimitParam] = "2",
            [TextMemoryPlugin.RelevanceParam] = "0.79",
            ["input"] = "Ask: where do I live?"
        });

 Console.WriteLine($"Answer: {result.GetValue<string>()}"); 

========= Example: Using Recall in a Semantic Function =========
Answer: ["I\u0027ve been living in Seattle since 2005","My family is from New York"]


## Remove a memory by key

In [55]:
Console.WriteLine("========= Example: Remove a memory by key =========");

var result = await kernel.RunAsync(memoryFunctions["Remove"], new ContextVariables()
        {
            [TextMemoryPlugin.CollectionParam] = MemoryCollectionName,
            [TextMemoryPlugin.KeyParam] = "info5"
        });
Console.WriteLine(result);

========= Example: Remove a memory by key =========
Microsoft.SemanticKernel.Orchestration.KernelResult


## Get a list of collections

In [56]:
Console.WriteLine("========= Example: Get a list of collections =========");
var collections = await textMemory.GetCollectionsAsync();
foreach (var collection in collections)
{
    Console.WriteLine(collection);
}

========= Example: Remove a memory by key =========
aboutMe


## Remove a memory using the textMemory functionality

In [57]:
Console.WriteLine("========= Example: Remove a memory by key =========");
await textMemory.RemoveAsync(MemoryCollectionName, "info1");


========= Example: Remove a memory by key =========
